In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout


In [2]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)

#Randomize the order of the input images
s=np.arange(data.shape[0])
np.random.seed(43)
np.random.shuffle(s)
data=data[s]
labels=labels[s]

Separando Dataset para treino e test

Converterndo para one hot encoding

In [3]:
#Spliting the images into train and validation sets
(X_train,X_val)=data[(int)(0.2*len(labels)):],data[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

#Using one hote encoding for the train and validation labels
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [4]:
# plt.figure()
# plt.imshow(X_train[70])
# plt.colorbar()
# plt.grid(False)
# plt.show()

Criando o modelo

In [5]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

Compilando o modelo e fazendo o Fit

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,validation_data=(X_val, y_val))
model.save("my_model.h5")

Epoch 1/15
981/981 [==============================] - 28s 28ms/step - loss: 1.1627 - accuracy: 0.6709 - val_loss: 0.1257 - val_accuracy: 0.9694
Epoch 2/15
981/981 [==============================] - 27s 27ms/step - loss: 0.2180 - accuracy: 0.9355 - val_loss: 0.0615 - val_accuracy: 0.9850
Epoch 3/15
981/981 [==============================] - 27s 27ms/step - loss: 0.1326 - accuracy: 0.9599 - val_loss: 0.0568 - val_accuracy: 0.9872
Epoch 4/15
981/981 [==============================] - 27s 27ms/step - loss: 0.1004 - accuracy: 0.9691 - val_loss: 0.0345 - val_accuracy: 0.9925
Epoch 5/15
981/981 [==============================] - 27s 28ms/step - loss: 0.0841 - accuracy: 0.9750 - val_loss: 0.0324 - val_accuracy: 0.9922
Epoch 6/15
981/981 [==============================] - 27s 28ms/step - loss: 0.0738 - accuracy: 0.9763 - val_loss: 0.0306 - val_accuracy: 0.9925
Epoch 7/15
981/981 [==============================] - 27s 27ms/step - loss: 0.0631 - accuracy: 0.9809 - val_loss: 0.0374 - val_accuracy:

Plotando em graficos


In [7]:
# plt.figure(0)
# plt.plot(history.history['accuracy'], label='training accuracy')
# plt.plot(history.history['val_accuracy'], label='val accuracy')
# plt.title('Accuracy')
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.legend()
# plt.show()

In [8]:
# plt.figure(1)
# plt.plot(history.history['loss'], label='training loss')
# plt.plot(history.history['val_loss'], label='val loss')
# plt.title('Loss')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.legend()
# plt.show()

Dando Load no modelo

In [4]:
model = load_model("my_model.h5")

Testando a acuracia com o dataset de test

In [5]:
y_test = pd.read_csv('input/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
predict_x=model.predict(X_test) 

y_test = pd.read_csv('input/Train.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
predict_x=model.predict(X_test) 
model.save('traffic_classifier.h5')

1226/1226 [==============================] - 6s 5ms/step
